# Which borough for a London restaurant

## Table of Contents

1. <a href="#item1">Import necessary Libraries</a>    
2. <a href="#item2">Use BeautifulSoup to pull out from of the HTML and XML files</a> 
3. <a href="#item2">Use geopy library to get the latitude and longitude values</a> 
4. <a href="#item2">Use Foursquare to get the top 100 venues from each Borough</a> 
5. <a href="#item2">Cluster Neighborhoods</a> 
6. <a href="#item2">Examine Clusters</a> 


### 1. Import necessary Libraries

In [128]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

from bs4 import BeautifulSoup # library for pulling data out of HTML and XML files
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

from sklearn.cluster import KMeans # import k-means from clustering stage

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors



print('Libraries imported.')

Libraries imported.


### 2. Use BeautifulSoup to pull out from of the HTML and XML files

In [2]:
# url
url = 'https://en.wikipedia.org/wiki/London_boroughs'
data_xml = requests.get(url).text
soup_postal_codes = BeautifulSoup(data_xml,'lxml')

# find the table in xml code
Table_boroughs = soup_postal_codes.find('table',{'class':'wikitable sortable'})

# instantiate the dataframe
column_names = ['Borough', 'Designation', 'Former areas 1', 'Former areas 2', 'Former areas 3', 'Former areas 4', 'Former areas 5'] 
Borough_table = pd.DataFrame(columns=column_names)

#Table_boroughs

Table_boroughs = Table_boroughs.find_all('tr')
boroughs_list = []

for balise_tr in Table_boroughs:
    balise_td = balise_tr.find_all('td')
    row = [i.text for i in balise_td]
    boroughs_list.append(row)

del boroughs_list[0]

for data in boroughs_list:
    Borough = data[0]
    Designation = data[1]
    Former_areas_1 = data[2]
    Former_areas_2 = data[3]
    Former_areas_3 = data[4]
    Former_areas_4 = data[5]
    Former_areas_5 = data[6]

    Borough_table = Borough_table.append({'Borough': Borough,
                                   'Designation': Designation,
                                   'Former areas 1':Former_areas_1,
                                    'Former areas 2':Former_areas_2,
                                    'Former areas 3':Former_areas_3,
                                    'Former areas 4':Former_areas_4,
                                    'Former areas 5':Former_areas_5 }, 
                                    ignore_index=True)

# clean data from :\n and [notes]
Borough_table.Borough = Borough_table.Borough.apply(lambda ele : ele.strip('\n'))
Borough_table.Designation = Borough_table.Designation.apply(lambda ele : ele.strip('\n'))

result_split = Borough_table['Borough'].str.split('[')


Borough_list = []
for ele in result_split:
    #print(ele[0])
    Borough_list.append(ele[0])

Borough_table['Borough'] = Borough_list

Borough_table_IO = Borough_table[['Borough','Designation']]



In [3]:
Borough_table_IO.head()

,Borough,Designation
0,Greenwich,Inner
1,Hackney,Inner
2,Hammersmith,Inner
3,Islington,Inner
4,Kensington and Chelsea,Inner


### 3. Use geopy library to get the latitude and longitude values

In [4]:
# function that extracts the coordinate of a city
def get_coordinate(address):
    
    geolocator = Nominatim(user_agent="explorer")
    location = geolocator.geocode(address)
    return location

In [123]:
 London_loc = get_coordinate( 'London, UK')

In [124]:
print('London_loc.latitude',London_loc.latitude)
print('London_loc.longitude',London_loc.longitude)

London_loc.latitude 51.5073219
London_loc.longitude -0.1276474


In [5]:
Borough_list_IO = []

for Borough in Borough_table_IO.Borough :
    LD_loc = get_coordinate(Borough + ', London, UK')
    row = [Borough, LD_loc.latitude,LD_loc.longitude]
    #print('row : ', row)
    Borough_list_IO.append(row)


In [6]:
# instantiate the dataframe

column_names = ['Borough', 'latitude', 'longitude'] 
Borough_lat_long = pd.DataFrame(columns=column_names)


In [7]:
for data in Borough_list_IO:
    Borough = data[0]
    latitude = data[1]
    longitude = data[2]


    Borough_lat_long = Borough_lat_long.append({'Borough': Borough,
                                                'latitude': latitude,
                                                'longitude':longitude}, 
                                               ignore_index=True)


In [8]:
Borough_lat_long.head()

,Borough,latitude,longitude
0,Greenwich,51.482084,-0.004542
1,Hackney,51.543240,-0.049362
2,Hammersmith,51.492038,-0.223640
3,Islington,51.538429,-0.099905
4,Kensington and Chelsea,51.498995,-0.199123


### 4. Use Foursquare to get the top 100 venues from each Borough

In [9]:
# The code was removed by Watson Studio for sharing.

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT= 100, VERSION = '20120618' ):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
london_venues = getNearbyVenues(names=Borough_lat_long['Borough'],
                                   latitudes=Borough_lat_long['latitude'],
                                   longitudes=Borough_lat_long['longitude']
                                  )

Greenwich
Hackney
Hammersmith
Islington
Kensington and Chelsea
Lambeth
Lewisham
Southwark
Tower Hamlets
Wandsworth
Westminster
Barking
Barnet
Bexley
Brent
Bromley
Croydon
Ealing
Enfield
Haringey
Harrow
Havering
Hillingdon
Hounslow
Kingston upon Thames
Merton
Newham
Redbridge
Richmond upon Thames
Sutton
Camden
Waltham Forest


In [12]:
london_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Greenwich,51.482084,-0.004542,Old Royal Naval College,51.483234,-0.005579,Historic Site
1,Greenwich,51.482084,-0.004542,Painted Hall,51.482889,-0.006420,Museum
2,Greenwich,51.482084,-0.004542,National Maritime Museum,51.481329,-0.005581,History Museum
3,Greenwich,51.482084,-0.004542,Greenwich Naval College Gardens,51.483007,-0.008362,Garden
4,Greenwich,51.482084,-0.004542,The Plume of Feathers,51.481945,-0.001126,Pub


### Let's find out how many unique categories can be curated from all the returned venues

In [13]:
#london_venues['Venue Category'].value_counts()
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))


There are 220 uniques categories.


### Analyze Each Neighborhood


In [14]:
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Borough'] = london_venues['Borough'] 

# move neighborhood column to the first column
Borough_pos = london_onehot.columns.get_loc("Borough")
fixed_columns = ([london_onehot.columns[Borough_pos]] + 
                 list(london_onehot.columns[:Borough_pos]) + 
                 list(london_onehot.columns[(Borough_pos+1):]))

london_onehot = london_onehot[fixed_columns]
london_onehot.head()

,Borough,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Greenwich,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Greenwich,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Greenwich,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Greenwich,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Greenwich,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Let's print each borough along with the top 5 most common venues


In [15]:
# Next, let's group rows by Borough and by taking the mean of the frequency of occurrence of each category
london_grouped = london_onehot.groupby('Borough').mean().reset_index()

num_top_venues = 5

for hood in london_grouped['Borough']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barking----
                  venue  freq
0              Platform  0.13
1         Grocery Store  0.13
2  Fast Food Restaurant  0.09
3           Supermarket  0.09
4                   Pub  0.04


----Barnet----
               venue  freq
0                Pub   0.4
1        Supermarket   0.2
2      Train Station   0.2
3                Gym   0.2
4  Afghan Restaurant   0.0


----Bexley----
                  venue  freq
0                   Pub  0.18
1  Fast Food Restaurant  0.18
2         Train Station  0.09
3    Italian Restaurant  0.09
4      Greek Restaurant  0.09


----Brent----
                  venue  freq
0     Indian Restaurant  0.17
1  Fast Food Restaurant  0.17
2             Nightclub  0.17
3  Gym / Fitness Center  0.17
4                   Pub  0.17


----Bromley----
                  venue  freq
0           Coffee Shop  0.13
1        Clothing Store  0.11
2                   Pub  0.07
3  Gym / Fitness Center  0.07
4           Pizza Place  0.04


----Camden----
                 

### Let's put that into a pandas dataframe

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [100]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = london_grouped['Borough']

for ind in np.arange(london_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking,Platform,Grocery Store,Fast Food Restaurant,Supermarket,Theater,Coffee Shop,Pizza Place,Plaza,Portuguese Restaurant,Pub
1,Barnet,Pub,Supermarket,Train Station,Gym,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
2,Bexley,Pub,Fast Food Restaurant,Italian Restaurant,Greek Restaurant,Breakfast Spot,Indian Restaurant,Toy / Game Store,Chinese Restaurant,Train Station,Yoga Studio
3,Brent,Pub,Indian Restaurant,Cupcake Shop,Fast Food Restaurant,Nightclub,Gym / Fitness Center,Yoga Studio,English Restaurant,Fish Market,Fish & Chips Shop
4,Bromley,Coffee Shop,Clothing Store,Gym / Fitness Center,Pub,Burger Joint,Pizza Place,Furniture / Home Store,Sushi Restaurant,Supermarket,Portuguese Restaurant


### 5. Cluster Neighborhoods

In [101]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Borough', 1) # borough_venues_sorted ,  london_grouped

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 0, 0, 4, 4, 4, 4, 4, 4], dtype=int32)

In [102]:
#london_onehot.head()
#kmeans.labels_[0:10] 

print('kmeans.labels_.shape : ', kmeans.labels_.shape)
print('borough_venues_sorted.shape : ', borough_venues_sorted.shape)
print('london_onehot.shape : ', london_onehot.shape)

kmeans.labels_.shape :  (32,)
borough_venues_sorted.shape :  (32, 11)
london_onehot.shape :  (1377, 221)


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
#rainbow = ['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']

# manhattan_merged = 

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [103]:
# add clustering labels
borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [117]:
london_venues_merged = Borough_lat_long

# merge toronto_grouped with toronto_data to add latitude/longitude for each Borough
london_venues_merged = london_venues_merged.join(borough_venues_sorted.set_index('Borough'), on='Borough')


In [118]:
london_venues_merged.head()

,Borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Greenwich,51.482084,-0.004542,4,Pub,Boat or Ferry,History Museum,Pier,Bar,Market,Bakery,Burger Joint,Art Gallery,Garden
1,Hackney,51.543240,-0.049362,4,Coffee Shop,Pub,Café,Yoga Studio,Theater,Boutique,Flea Market,Movie Theater,Butcher,Sporting Goods Shop
2,Hammersmith,51.492038,-0.223640,4,Pub,Café,Coffee Shop,Hotel,Gym / Fitness Center,Sandwich Place,Clothing Store,Pharmacy,Grocery Store,Portuguese Restaurant
3,Islington,51.538429,-0.099905,4,Pub,Mediterranean Restaurant,French Restaurant,Burger Joint,Bakery,Gastropub,Café,Furniture / Home Store,Theater,Mexican Restaurant
4,Kensington and Chelsea,51.498995,-0.199123,4,Café,Italian Restaurant,Modern European Restaurant,Pub,Clothing Store,Supermarket,Cupcake Shop,Bakery,Burger Joint,Japanese Restaurant


In [129]:
# create map
map_clusters = folium.Map(location=[London_loc.latitude, London_loc.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# manhattan_merged = london_venues_merged

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_venues_merged['latitude'], london_venues_merged['longitude'], london_venues_merged['Borough'], london_venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [121]:
#borough_venues_sorted.head()

### 6. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [122]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 0, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Barnet,Pub,Supermarket,Train Station,Gym,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
2,Bexley,Pub,Fast Food Restaurant,Italian Restaurant,Greek Restaurant,Breakfast Spot,Indian Restaurant,Toy / Game Store,Chinese Restaurant,Train Station,Yoga Studio
3,Brent,Pub,Indian Restaurant,Cupcake Shop,Fast Food Restaurant,Nightclub,Gym / Fitness Center,Yoga Studio,English Restaurant,Fish Market,Fish & Chips Shop
15,Hillingdon,Pub,Fast Food Restaurant,Chinese Restaurant,Park,Yoga Studio,English Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant
23,Newham,Pub,Fast Food Restaurant,Café,Yoga Studio,English Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Farmers Market


#### Cluster 2

In [88]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 1, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Redbridge,Hotel,Pizza Place,Pub,Metro Station,Eastern European Restaurant,Event Space,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant


#### Cluster 3

In [89]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 2, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Richmond upon Thames,Bus Station,Garden Center,Trail,German Restaurant,Yoga Studio,English Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant


#### Cluster 4

In [90]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 3, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Sutton,Bakery,Grocery Store,Park,Train Station,Italian Restaurant,Falafel Restaurant,Event Space,Exhibit,Yoga Studio,English Restaurant


#### Cluster 5

In [91]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 4, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking,Platform,Grocery Store,Fast Food Restaurant,Supermarket,Theater,Coffee Shop,Pizza Place,Plaza,Portuguese Restaurant,Pub
4,Bromley,Coffee Shop,Clothing Store,Gym / Fitness Center,Pub,Burger Joint,Pizza Place,Furniture / Home Store,Sushi Restaurant,Supermarket,Portuguese Restaurant
5,Camden,Pub,Coffee Shop,Market,Vegetarian / Vegan Restaurant,Café,Italian Restaurant,Ice Cream Shop,Shoe Store,Bar,Clothing Store
6,Croydon,Pub,Coffee Shop,Asian Restaurant,Nightclub,Gaming Cafe,Sushi Restaurant,Malay Restaurant,Burger Joint,Spanish Restaurant,Clothing Store
7,Ealing,Coffee Shop,Pub,Clothing Store,Platform,Bakery,Burger Joint,Italian Restaurant,Park,Café,Grocery Store
8,Enfield,Coffee Shop,Clothing Store,Supermarket,Sandwich Place,Café,Optical Shop,Gym / Fitness Center,Shopping Mall,Women's Store,Bookstore
9,Greenwich,Pub,Boat or Ferry,History Museum,Pier,Bar,Market,Bakery,Burger Joint,Art Gallery,Garden
10,Hackney,Coffee Shop,Pub,Café,Yoga Studio,Theater,Boutique,Flea Market,Movie Theater,Butcher,Sporting Goods Shop
11,Hammersmith,Pub,Café,Coffee Shop,Hotel,Gym / Fitness Center,Sandwich Place,Clothing Store,Pharmacy,Grocery Store,Portuguese Restaurant
12,Haringey,Bus Stop,Coffee Shop,Park,Café,Indian Restaurant,Platform,Sandwich Place,Bulgarian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant


#### Cluster 6

In [92]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 5, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 7

In [75]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 6, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Richmond upon Thames,Bus Station,Garden Center,Trail,German Restaurant,Yoga Studio,English Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant


#### Cluster 8

In [76]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 7, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Brent,Pub,Indian Restaurant,Cupcake Shop,Fast Food Restaurant,Nightclub,Gym / Fitness Center,Yoga Studio,English Restaurant,Fish Market,Fish & Chips Shop


#### Cluster 9

In [77]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 8, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Sutton,Bakery,Grocery Store,Park,Train Station,Italian Restaurant,Falafel Restaurant,Event Space,Exhibit,Yoga Studio,English Restaurant


#### Cluster 10

In [78]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 9, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Bexley,Pub,Fast Food Restaurant,Italian Restaurant,Greek Restaurant,Breakfast Spot,Indian Restaurant,Toy / Game Store,Chinese Restaurant,Train Station,Yoga Studio


#### Cluster 11

In [79]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 10, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking,Platform,Grocery Store,Fast Food Restaurant,Supermarket,Theater,Coffee Shop,Pizza Place,Plaza,Portuguese Restaurant,Pub
4,Bromley,Coffee Shop,Clothing Store,Gym / Fitness Center,Pub,Burger Joint,Pizza Place,Furniture / Home Store,Sushi Restaurant,Supermarket,Portuguese Restaurant
8,Enfield,Coffee Shop,Clothing Store,Supermarket,Sandwich Place,Café,Optical Shop,Gym / Fitness Center,Shopping Mall,Women's Store,Bookstore
12,Haringey,Bus Stop,Coffee Shop,Park,Café,Indian Restaurant,Platform,Sandwich Place,Bulgarian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant
16,Hounslow,Coffee Shop,Hotel,Fast Food Restaurant,Clothing Store,Indian Restaurant,Grocery Store,Café,Chinese Restaurant,Sandwich Place,Bakery
18,Kensington and Chelsea,Café,Italian Restaurant,Modern European Restaurant,Pub,Clothing Store,Supermarket,Cupcake Shop,Bakery,Burger Joint,Japanese Restaurant
21,Lewisham,Clothing Store,Coffee Shop,Fast Food Restaurant,Platform,Supermarket,Pharmacy,Pub,Grocery Store,Restaurant,Café
28,Tower Hamlets,Park,Grocery Store,Indian Restaurant,Sushi Restaurant,Bus Stop,Sandwich Place,Harbor / Marina,Convenience Store,Bar,Café
29,Waltham Forest,Fast Food Restaurant,Fried Chicken Joint,Coffee Shop,Pub,Clothing Store,Café,Platform,Greek Restaurant,Supermarket,Pharmacy
30,Wandsworth,Pub,Clothing Store,Coffee Shop,Burger Joint,Pizza Place,Asian Restaurant,Supermarket,Gym / Fitness Center,Restaurant,Bookstore


#### Cluster 12

In [80]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 11, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Merton,Tram Station,Pub,Brazilian Restaurant,Thai Restaurant,Flea Market,Farm,Park,Dessert Shop,Hardware Store,Cricket Ground


#### Cluster 13

In [81]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 12, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


#### Cluster 14

In [119]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 13, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Haringey,Bus Stop,Coffee Shop,Park,Café,Indian Restaurant,Platform,Sandwich Place,Bulgarian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant
16,Hounslow,Coffee Shop,Hotel,Fast Food Restaurant,Clothing Store,Indian Restaurant,Grocery Store,Café,Chinese Restaurant,Sandwich Place,Bakery
21,Lewisham,Clothing Store,Coffee Shop,Fast Food Restaurant,Platform,Supermarket,Pharmacy,Pub,Grocery Store,Restaurant,Café
29,Waltham Forest,Fast Food Restaurant,Fried Chicken Joint,Coffee Shop,Pub,Clothing Store,Café,Platform,Greek Restaurant,Supermarket,Pharmacy


#### Cluster 15

In [120]:
borough_venues_sorted.loc[borough_venues_sorted['Cluster Labels'] == 14, borough_venues_sorted.columns[[1] + list(range(2, borough_venues_sorted.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


When we examine the clusters , the most common venue appear to be the Pub.  It's obvious in the cluster 1 and in cluster  